In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bipolar_disorder/GSE120340'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Aberrant transcriptomes and DNA methylomes define pathways that drive pathogenesis and loss of brain laterality/asymmetry in schizophrenia and bipolar disorder [Affymetrix]"
!Series_summary	"Although the loss or reversal of brain laterality is one of the most consistent modalities in schizophrenia (SCZ) and bipolar disorder (BD), its molecular basis remains elusive. Our limited previous studies indicated that epigenetic modifications are key to the asymmetric transcriptomes of brain hemispheres. We used whole-genome expression microarrays to profile post-mortem brain samples from subjects with SCZ, psychotic BD [BD(+)] or non-psychotic BD [BD(-)], or matched controls (n=10/group, corresponding to different brain hemispheres) and performed whole-genome DNA methylation (DNAM) profiling of the same samples (n=3-4/group) to identify pathways associated with SCZ or BD(+) and genes/sites susceptible to epigenetic regulation. qRT-PCR and quantitative DNA

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = None
gender_row = None

def convert_trait(value):
    if ':' in value:
        value = value.split(':')[1].strip().lower()
        if value == 'control':
            return 0
        elif value in ['scz', 'bd(-)', 'bd(+)']:
            return 1
    return None

convert_age = convert_gender = None

save_cohort_info('GSE120340', './preprocessed/Bipolar_disorder/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Bipolar_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Bipolar_disorder/trait_data/GSE120340.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3398477': [0], 'GSM3398478': [0], 'GSM3398479': [0], 'GSM3398480': [0], 'GSM3398481': [0], 'GSM3398482': [0], 'GSM3398483': [0], 'GSM3398484': [0], 'GSM3398485': [0], 'GSM3398486': [0], 'GSM3398487': [1], 'GSM3398488': [1], 'GSM3398489': [1], 'GSM3398490': [1], 'GSM3398491': [1], 'GSM3398492': [1], 'GSM3398493': [1], 'GSM3398494': [1], 'GSM3398495': [1], 'GSM3398496': [1], 'GSM3398497': [1], 'GSM3398498': [1], 'GSM3398499': [1], 'GSM3398500': [1], 'GSM3398501': [1], 'GSM3398502': [1], 'GSM3398503': [1], 'GSM3398504': [1], 'GSM3398505': [1], 'GSM3398506': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifier and gene symbol keys.
identifier_key = 'ID'
gene_symbol_key = 'Description'

# 2. Get the dataframe storing the mapping between probe IDs and genes.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library.
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Debugging prints
print("Gene mapping dataframe created:")
print(preview_df(mapping_df))

print("Gene data after applying gene mapping:")
print(preview_df(gene_data))


Gene mapping dataframe created:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'Gene': [['alpha-1-B glycoprotein'], ['N-acetyltransferase 2 ', 'arylamine N-acetyltransferase', ''], ['adenosine deaminase'], ['cadherin 2', ' type 1', ' N-cadherin ', 'neuronal', ''], ['v-akt murine thymoma viral oncogene homolog 3 ', 'protein kinase B', ' gamma', '']]}
Gene data after applying gene mapping:
{'GSM3398477': [6.277373126660078, 6.767598610363636, 7.527698902955556, 6.116360897833334, 9.302105121764706], 'GSM3398478': [6.243025334789623, 6.899679454303031, 7.510344182544444, 6.231022517166667, 9.23272963082353], 'GSM3398479': [6.229758028323332, 6.615911690060606, 7.493422205288889, 6.082252319833334, 9.368271839941176], 'GSM3398480': [6.255079223324744, 6.582985092242424, 7.477973985933333, 5.927754006833333, 9.587786642588235], 'GSM3398481': [6.2867930136434875, 6.6518537523030306, 7.479553558666666, 6.1326713285, 9.32495826617647], 'GSM3398482': [6.240543012963643, 6.7667077974

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Bipolar_disorder/gene_data/GSE120340.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Bipolar_disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE120340', './preprocessed/Bipolar_disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Bipolar_disorder/GSE120340.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Bipolar_disorder', the least common label is '0.0' with 10 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Bipolar_disorder' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Bipolar_disorder/cohort_info.json
